# MSGF VS PeptideAtlas Reanalysis

#### reqired files (put under same folder):
###### 1 all_usi.txt : all peptide atlas usi found
###### 2 MassIVE-KB_HPP_proteins.tsv: all MassIVE proteins
###### 3 all ambituity files on same directory eg. MSV000096271
###### 4 fasta_file = 'uniprotkb_human_proteome_UP000005640_with_isoforms_2024-10-08.fasta'
###### 5 PA_observations.csv peptide atlas all peptides
###### 6 dictionary file Human_proteome_dictionary_I_replaced_by_L.pkl with I subsituted by L


#### Merge ambiguity files --> ambiguity_merged.tsv

In [ ]:
import os
import shutil
import pandas as pd

# Directory containing the TSV files
directory = 'D:/ccms/Protein_id_reanalysis/Evaluate_reanalysis/MSV000083508' #TODO

# Output file path
output_file = os.path.join(directory, 'ambiguity_merged.tsv')

# Archive directory
archive_directory = os.path.join(directory, 'ambiguity_archive')

# Create the archive directory if it doesn't exist
if not os.path.exists(archive_directory):
    os.makedirs(archive_directory)

# Remove the output file if it already exists
# if os.path.exists(output_file):
#     os.remove(output_file)

# Iterate over files in the directory
for filename in os.listdir(directory):
    if filename.startswith('MSGF-PLUS-AMBIGUITY') and filename.endswith('.tsv'):
        print(f'Processing file: {filename}')
        filepath = os.path.join(directory, filename)
        try:
            df = pd.read_csv(filepath, sep='\t', low_memory=False)
        except pd.errors.EmptyDataError:
            print(f'Skipping empty file: {filename}')
            continue
        # Append to the output file
        df.to_csv(output_file, sep='\t', index=False, mode='a', header=not os.path.exists(output_file))
        # Move the processed file to the archive directory
        shutil.move(filepath, os.path.join(archive_directory, filename))

Processing file: MSGF-PLUS-AMBIGUITY-1dbb1e7a-group_by_spectrum-main.tsv
Processing file: MSGF-PLUS-AMBIGUITY-23297dcc-group_by_spectrum-main.tsv
Processing file: MSGF-PLUS-AMBIGUITY-2def881b-group_by_spectrum-main.tsv
Processing file: MSGF-PLUS-AMBIGUITY-3acae0d9-group_by_spectrum-main.tsv
Processing file: MSGF-PLUS-AMBIGUITY-5df4308f-group_by_spectrum-main.tsv
Processing file: MSGF-PLUS-AMBIGUITY-641370a3-group_by_spectrum-main.tsv
Processing file: MSGF-PLUS-AMBIGUITY-6885751f-group_by_spectrum-main.tsv
Processing file: MSGF-PLUS-AMBIGUITY-6972734d-group_by_spectrum-main.tsv
Processing file: MSGF-PLUS-AMBIGUITY-6af6149e-group_by_spectrum-main.tsv
Processing file: MSGF-PLUS-AMBIGUITY-6f0e5be2-group_by_spectrum-main.tsv
Processing file: MSGF-PLUS-AMBIGUITY-7803f3a8-group_by_spectrum-main.tsv
Processing file: MSGF-PLUS-AMBIGUITY-81c2b464-group_by_spectrum-main.tsv
Processing file: MSGF-PLUS-AMBIGUITY-8ce26958-group_by_spectrum-main.tsv
Processing file: MSGF-PLUS-AMBIGUITY-955e3b5a-group

#### ambiguity_merged.tsv --> filtered.tsv

In [3]:
import pandas as pd

# Load the datasets in chunks
chunksize = 10000
proteins_to_skip_file = 'MassIVE-KB_HPP_proteins.tsv'
with open(proteins_to_skip_file, 'r') as file:
    massive_kb_proteins = set(line.strip() for line in file)

filtered_chunks = []
for chunk in pd.read_csv('ambiguity_merged.tsv', sep='\t', low_memory=False, error_bad_lines=False, chunksize=chunksize):
    # Process the 'opt_global_TopCanonicalProtein' column to extract the protein ID

    chunk['opt_global_TopCanonicalProtein'] = chunk['opt_global_TopCanonicalProtein'].apply(
        lambda x: x.split('|')[1] if isinstance(x, str) and '|' in x else x
    )
    # Filter the chunk
    filtered_chunk = chunk[~chunk['opt_global_TopCanonicalProtein'].isin(massive_kb_proteins)]
    filtered_chunks.append(filtered_chunk)

# Concatenate all filtered chunks
filtered_reanalysis_df = pd.concat(filtered_chunks)

# Save the filtered dataframe to a new file
filtered_reanalysis_df.to_csv('filtered.tsv', sep='\t', index=False)

C:\Users\xuech\AppData\Local\Temp\ipykernel_34688\1751162628.py:10: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  for chunk in pd.read_csv('ambiguity_merged.tsv', sep='\t', low_memory=False, error_bad_lines=False, chunksize=chunksize):


## spectrum level

#### all_usi.txt + filtered.tsv --> example_reanalysis_spectrum.tsv

In [ ]:
import pandas as pd
import re
import time

# Read the PeptideAtlas USIs from merged.txt
usi_file = 'all_usi.txt'
usi_data = []
with open(usi_file, 'r') as file:
    for line in file:
        parts = line.strip().split(':')
        if len(parts) == 6:
            dataset, spectrum_file, scan, peptide_identification, peptide_charge = parts[1], parts[2], parts[4], parts[5].split('/')[0], parts[5].split('/')[1]
            usi_data.append([line.strip(), dataset, spectrum_file, scan, peptide_identification, peptide_charge])

usi_df = pd.DataFrame(usi_data, columns=['USI', 'Dataset', 'Spectrum_File', 'Scan_Number', 'Peptide_Identification', 'Peptide_Charge'])
usi_df.to_csv('df_print.tsv', sep='\t', index=False)

# Create a dictionary for quick lookup
usi_dict = {}
for _, row in usi_df.iterrows():
    key = (row['Spectrum_File'], row['Scan_Number'])
    usi_dict[key] = row

# Read the reanalysis results from MSGF-PLUS-AMBIGUITY-81a33a88-group_by_spectrum-main.tsv
reanalysis_file = 'filtered.tsv'
reanalysis_df = pd.read_csv(reanalysis_file, sep='\t', low_memory=False)

# Initialize columns for the output DataFrame
reanalysis_df.insert(0, 'PeptideAtlas_USI', '')
reanalysis_df.insert(1, 'PeptideAtlas_peptide', '')
reanalysis_df.insert(2, 'PeptideAtlas_peptide_demod', '')
reanalysis_df.insert(3, 'Peptide_match', '')
reanalysis_df.insert(4, 'PeptideAtlas_charge', '')


# Match spectra from the PeptideAtlas USIs lists to the reanalysis results
all_MSGF_spectrum_files = set()
    # Precompute set of all spectrum_file names in usi_dict for fast lookup
if 'usi_spectrum_files_set' not in globals():
    usi_spectrum_files_set = {k[0] for k in usi_dict.keys()}


for index, row in reanalysis_df.iterrows():
    original_filepath = row['opt_global_OriginalFilepath']
    scan_number = str(row['opt_global_scan'])
    
    # Extract the spectrum file name from the original file path and remove the .mzML extension if present
    spectrum_file = original_filepath.split('/')[-1].replace('.mzML', '')
    
    key = (spectrum_file, scan_number)

    if spectrum_file in usi_spectrum_files_set:
        all_MSGF_spectrum_files.add(spectrum_file)
        
    if key in usi_dict:
        usi_row = usi_dict[key]
        reanalysis_df.at[index, 'PeptideAtlas_USI'] = usi_row['USI']
        reanalysis_df.at[index, 'PeptideAtlas_peptide'] = usi_row['Peptide_Identification']
        # Remove all substrings like "[*]" from PeptideAtlas_peptide
        peptide_demod = re.sub(r'\[.*?\]', '', usi_row['Peptide_Identification']).replace('-', '')
        reanalysis_df.at[index, 'PeptideAtlas_peptide_demod'] = peptide_demod
        
        # Set Peptide_match to 1 if PeptideAtlas_peptide_demod matches opt_global_UnmodPep, otherwise 0
        reanalysis_df.at[index, 'Peptide_match'] = 1 if peptide_demod == row['opt_global_UnmodPep'] else 0
        reanalysis_df.at[index, 'PeptideAtlas_charge'] = usi_row['Peptide_Charge']
        #print(f'Matched file: {usi_row["Spectrum_File"]}, Scan number: {usi_row["Scan_Number"]}')

    #elif spectrum_file is in usi_dict but scan_number is not in usi_dict, still add the spectrum file to all_MSGF_spectrum_files
    if index % 1000 == 0:
        if index == 0:
            start_time = time.time()
        else:
            elapsed = time.time() - start_time
            percent = (index + 1) / len(reanalysis_df) * 100
            total_est = elapsed / (index + 1) * len(reanalysis_df)
            remaining = total_est - elapsed
            hrs, rem = divmod(remaining, 3600)
            mins, secs = divmod(rem, 60)
            print(f'Processed {index} rows ({percent:.2f}%). Estimated time left: {int(hrs)}h {int(mins)}m {int(secs)}s')

    # Identify the datasets and spectrum files that have at least one match
    # matched_datasets = set()
matched_spectrum_files = set()
matched_scans = set()

for index, row in reanalysis_df.iterrows():

    if row['PeptideAtlas_USI']:
        usi_parts = row['PeptideAtlas_USI'].split(':')
        if len(usi_parts) == 6:
            dataset = usi_parts[1].replace('.mzML', '')
            spectrum_file = usi_parts[2]
            matched_spectrum_files.add(spectrum_file)
            matched_scans.add((spectrum_file, usi_parts[4]))
print(f'Matched {len(matched_spectrum_files)} spectrum files and {len(matched_scans)} scans.')

# Add empty USIs for unmatched spectra all at once
new_rows = []
for key, usi_row in usi_dict.items():
    spectrum_file, scan_number = key
    if (usi_row['Dataset'] == 'MSV000083508' or usi_row['Dataset'] == 'PXD010154') and spectrum_file in all_MSGF_spectrum_files and key not in matched_scans: #TODO
        new_rows.append({
            'PeptideAtlas_USI': usi_row['USI'],
            'PeptideAtlas_peptide': usi_row['Peptide_Identification'],
            'PeptideAtlas_peptide_demod': re.sub(r'\[.*?\]', '', usi_row['Peptide_Identification']).replace('-', ''),
            'Peptide_match': 0,
            'PeptideAtlas_charge': usi_row['Peptide_Charge'],
            'opt_global_OriginalFilepath': '',
            'opt_global_scan': '',
            'opt_global_UnmodPep': ''
        })

# Append all new rows to the DataFrame at once
if new_rows:
    reanalysis_df = pd.concat([reanalysis_df, pd.DataFrame(new_rows)], ignore_index=True)


output_file = 'example_reanalysis_spectrum.tsv'
reanalysis_df.to_csv(output_file, sep='\t', index=False)

Processed 1000 rows (0.15%). Estimated time left: 0h 0m 35s
Processed 2000 rows (0.29%). Estimated time left: 0h 0m 39s
Processed 3000 rows (0.44%). Estimated time left: 0h 0m 40s
Processed 4000 rows (0.59%). Estimated time left: 0h 0m 42s
Processed 5000 rows (0.74%). Estimated time left: 0h 0m 41s
Processed 6000 rows (0.88%). Estimated time left: 0h 0m 41s
Processed 7000 rows (1.03%). Estimated time left: 0h 0m 41s
Processed 8000 rows (1.18%). Estimated time left: 0h 0m 42s
Processed 9000 rows (1.32%). Estimated time left: 0h 0m 41s
Processed 10000 rows (1.47%). Estimated time left: 0h 0m 42s
Processed 11000 rows (1.62%). Estimated time left: 0h 0m 41s
Processed 12000 rows (1.76%). Estimated time left: 0h 0m 41s
Processed 13000 rows (1.91%). Estimated time left: 0h 0m 43s
Processed 14000 rows (2.06%). Estimated time left: 0h 0m 42s
Processed 15000 rows (2.21%). Estimated time left: 0h 0m 41s
Processed 16000 rows (2.35%). Estimated time left: 0h 0m 41s
Processed 17000 rows (2.50%). Est

: 

In [ ]:
# # Filter rows where '#SpecFile' contains '01321_D12_P013127_S00_N12_R1'
# filtered_rows = spectrum_df[spectrum_df['#SpecFile'].str.contains('01321_D12_P013127_S00_N12_R1', na=False)]

# # Get the set of scan numbers from the 'opt_global_scan' column
# scan_numbers = set(filtered_rows['opt_global_scan'].dropna().astype(int))

# print(sorted(scan_numbers))

[1948, 2643, 2682, 2824, 2968, 3251, 3883, 4000, 4015, 4405, 4899, 4941, 4966, 4979, 5189, 5227, 5457, 5496, 5664, 5767, 5897, 5925, 6006, 6147, 6182, 6360, 6731, 6735, 6745, 6972, 7551, 7853, 7989, 8188, 8864, 8871, 9453, 10217, 10333, 10445, 10707, 10959, 11576, 11643, 11860, 12044, 12248, 12417, 12530, 12894, 13018, 13164, 13255, 13436, 13569, 13609, 13680, 13732, 13785, 13907, 13927, 13966, 14025, 14090, 14294, 14377, 14462, 14523, 14557, 14561, 14672, 14726, 14782, 14805, 14827, 14878, 15080, 15338, 15494, 15879, 15928, 16102, 16128, 16225, 16275, 16534, 16807, 16840, 16849, 16881, 16922, 16947, 17131, 17245, 17330, 17363, 17399, 17871, 17903, 17992, 18214, 18769, 18801, 18829, 19234, 19254, 19640, 19690, 19749, 19795, 20355, 20690, 20697, 20956, 20981, 21195, 21286, 21378, 21594, 21643, 21736, 21812, 21871, 21905, 21911, 21931, 21960, 22238, 22255, 22277, 22600, 22681, 22758, 22901, 22914, 22916, 23211, 23300, 23308, 23361, 23435, 23630, 23785, 23828, 24021, 24089, 24096, 24414, 

In [ ]:
# usi_file = 'all_usi.txt'
# usi_data = []
# with open(usi_file, 'r') as file:
#     for line in file:
#         parts = line.strip().split(':')
#         if len(parts) == 6:
#             dataset, spectrum_file, scan, peptide_identification, peptide_charge = parts[1], parts[2], parts[4], parts[5].split('/')[0], parts[5].split('/')[1]
#             usi_data.append([line.strip(), dataset, spectrum_file, scan, peptide_identification, peptide_charge])
#             # Print the first 5 examples from usi_data
# # Print the first 5 examples from usi_data
# for example in usi_data[:5]:
#     print(example)

# # Print the example that matches the specific USI
# target_usi = "mzspec:PXD010154:01321_D12_P013127_S00_N12_R1:scan:13514:FNSRTAELLSHHQVEIK/4"
# for example in usi_data:
#     if example[0] == target_usi:
#         print("Matched example:", example)
#         break

['mzspec:PXD015899:CPTAC_OvC_JB5429_iTRAQ_18_4Apr12_Cougar_12-03-22:scan:8441:[iTRAQ4plex]-LLIYGGSTR/2', 'PXD015899', 'CPTAC_OvC_JB5429_iTRAQ_18_4Apr12_Cougar_12-03-22', '8441', '[iTRAQ4plex]-LLIYGGSTR', '2']
['mzspec:PXD015899:CPTAC_OvC_JB5429_iTRAQ_18_4Apr12_Cougar_12-03-22:scan:8441:[iTRAQ4plex114]-LLIYGGSTR/2', 'PXD015899', 'CPTAC_OvC_JB5429_iTRAQ_18_4Apr12_Cougar_12-03-22', '8441', '[iTRAQ4plex114]-LLIYGGSTR', '2']
['mzspec:PXD019910:20180329_EMIF_tryp_s35_f03:scan:11092:[TMT6plex]-LLIYGGSTR/2', 'PXD019910', '20180329_EMIF_tryp_s35_f03', '11092', '[TMT6plex]-LLIYGGSTR', '2']
['mzspec:PXD019910:20180329_EMIF_tryp_s44_f03:scan:11199:[TMT6plex]-LLIYGGSTR/2', 'PXD019910', '20180329_EMIF_tryp_s44_f03', '11199', '[TMT6plex]-LLIYGGSTR', '2']
['mzspec:PXD006463:MS161140-B-20_161024053838:scan:18934:[TMT6plex]-LLIYGGSTR/2', 'PXD006463', 'MS161140-B-20_161024053838', '18934', '[TMT6plex]-LLIYGGSTR', '2']
Matched example: ['mzspec:PXD010154:01321_D12_P013127_S00_N12_R1:scan:13514:FNSRTAELLSH

## Peptide Level

#### example_reanalysis_spectrum.tsv -> example_reanalysis_peptide.tsv

In [ ]:
#multiprocessing doeson't work here, please directly call compare_reanalyze_peptide.py in terminal or copy and run code below

# import pandas as pd
# from Bio import SeqIO
# from multiprocessing import Pool, cpu_count
# import os

# # Load the data
# chunk_size = 10000
# fasta_file = 'uniprotkb_human_proteome_UP000005640_with_isoforms_2024-10-08.fasta'

# # Define a function to get the peptide sequence
# def get_peptide_sequence(row):
#     return row['PeptideAtlas_peptide_demod'] if pd.notna(row['PeptideAtlas_peptide_demod']) else row['opt_global_UnmodPep']

# # Define a function to get the peptide charge
# def get_peptide_charge(row):
#     return row['PeptideAtlas_charge'] if pd.notna(row['PeptideAtlas_charge']) else row['charge']

# # Define a function to get the protein ID from the sequence column "tr|D9J307|D9J307_HUMAN"
# def get_protein_id_from_msgf(row):
#     accession = row['accession']
#     if isinstance(accession, str) and '|' in accession:
#         parts = accession.split('|')
#         if len(parts) > 1:
#             return parts[1]
#     return None

# # Load protein sequences from fasta file
# # Pre-compute and store sequences in a dictionary for faster lookups
# protein_sequences = {}
# for record in SeqIO.parse(fasta_file, "fasta"):
#     gene_id = next((part.split('=')[1] for part in record.description.split() if part.startswith('GN=')), 'UNKNOWN')
#     protein_sequences[record.id] = {
#         'gene_id': gene_id,
#         'sequence': str(record.seq).replace('I', 'L')
#     }

# # Function to count matches in protein sequences
# def count_matches(peptide, allow_mutation=False):
#     peptide = peptide.replace('I', 'L')
#     protein_ids = set()
#     gene_ids = set()
#     protein_list = []
#     gene_list = []
    
#     for header, data in protein_sequences.items():
#         gene_id = data['gene_id']
#         sequence = data['sequence']
#         protein_id = header.split('|')[1] if '|' in header else header
        
#         if allow_mutation:
#             # Check for near-matches (SAAP)
#             for i in range(len(sequence) - len(peptide) + 1):
#                 window = sequence[i:i+len(peptide)]
#                 if sum(1 for a, b in zip(peptide, window) if a != b) <= 1:
#                     protein_ids.add(protein_id)
#                     gene_ids.add(gene_id)
#                     break
#         else:
#             # Check for exact matches
#             if peptide in sequence:
#                 protein_ids.add(protein_id)
#                 gene_ids.add(gene_id)
    
#     # Return counts and lists
#     return (
#         len(protein_ids), 
#         len(gene_ids), 
#         ';'.join(protein_ids) if protein_ids else 'None', 
#         ';'.join(gene_ids) if gene_ids else 'UNKNOWN'
#     )

# # Function to process a peptide
# def process_peptide(to_parallel_process):
#     peptide, peptide_data, peptideatlas_df = to_parallel_process
    
#     # Get exact matches and SAAP matches
#     num_proteins, num_genes, list_proteins, list_genes = count_matches(peptide)
#     num_proteins_saap, num_genes_saap, list_proteins_saap, list_genes_saap = count_matches(peptide, allow_mutation=True)
    
#     # Build the output row
#     peptide_row = {
#         'Peptide sequence': peptide,
#         'Peptide charge': peptide_data.apply(get_peptide_charge, axis=1).iloc[0],
#         'Protein identifier': peptide_data.apply(get_protein_id_from_msgf, axis=1).iloc[0],
#         'Num_specs_both': len(peptide_data[(pd.notna(peptide_data['PeptideAtlas_USI'])) & (pd.notna(peptide_data['sequence']))]),
#         'Num_specs_MSGF': len(peptide_data[(pd.isna(peptide_data['PeptideAtlas_USI'])) & (pd.notna(peptide_data['sequence']))]),
#         'Num_specs_PA': len(peptide_data[(pd.notna(peptide_data['PeptideAtlas_USI'])) & (pd.isna(peptide_data['sequence']))]),
#         'PA_peptide': 1 if not peptideatlas_df[peptideatlas_df.iloc[:, 5] == peptide].empty else 0,
#         'PA_psms': len(peptideatlas_df[peptideatlas_df.iloc[:, 5] == peptide]),
#         'Num_proteins': num_proteins,
#         'List_proteins': list_proteins,
#         'Num_genes': num_genes,
#         'List_genes': list_genes,
#         'Num_proteins_saap': num_proteins_saap,
#         'List_proteins_saap': list_proteins_saap,
#         'Num_genes_saap': num_genes_saap,
#         'List_genes_saap': list_genes_saap
#     }
#     #print(peptide_row)
#     return peptide_row
#     # Function to process an existing peptide
# def process_existing_peptide(peptide, peptide_data, peptideatlas_df):
#     # Build the output row without counting matches
#     peptide_row = {
#         'Peptide sequence': peptide,
#         'Peptide charge': peptide_data.apply(get_peptide_charge, axis=1).iloc[0],
#         'Protein identifier': peptide_data.apply(get_protein_id_from_msgf, axis=1).iloc[0],
#         'Num_specs_both': len(peptide_data[(pd.notna(peptide_data['PeptideAtlas_USI'])) & (pd.notna(peptide_data['sequence']))]),
#         'Num_specs_MSGF': len(peptide_data[(pd.isna(peptide_data['PeptideAtlas_USI'])) & (pd.notna(peptide_data['sequence']))]),
#         'Num_specs_PA': len(peptide_data[(pd.notna(peptide_data['PeptideAtlas_USI'])) & (pd.isna(peptide_data['sequence']))]),
#         'PA_peptide': 1 if not peptideatlas_df[peptideatlas_df.iloc[:, 5] == peptide].empty else 0,
#         'PA_psms': len(peptideatlas_df[peptideatlas_df.iloc[:, 5] == peptide]),
#         'Num_proteins': None,
#         'List_proteins': None,
#         'Num_genes': None,
#         'List_genes': None,
#         'Num_proteins_saap': None,
#         'List_proteins_saap': None,
#         'Num_genes_saap': None,
#         'List_genes_saap': None
#     }
#     print(f"Existing Peptide ID: {peptide_row['Peptide sequence']}")
#     return peptide_row
# # Load the PeptideAtlas data
# peptideatlas_df = pd.read_csv('PeptideAtlas_peptides.tsv', sep='\t')

# # Use a set to keep track of unique peptides
# unique_peptides_set = set()

# if __name__ == "__main__":
#     # Open the output file
#     peptide_file_exists = os.path.exists('example_reanalysis_peptide.tsv')
#     with open('example_reanalysis_peptide.tsv', 'a+') as output_file:
#         # Write the header
#         if not peptide_file_exists:
#             output_file.write('\t'.join([
#                 'Peptide sequence', 'Peptide charge', 'Protein identifier', 'Num_specs_both', 'Num_specs_MSGF', 'Num_specs_PA',
#                 'PA_peptide', 'PA_psms', 'Num_proteins', 'List_proteins', 'Num_genes', 'List_genes', 'Num_proteins_saap',
#                 'List_proteins_saap', 'Num_genes_saap', 'List_genes_saap'
#             ]) + '\n')
        
#         # Read the spectrum file in chunks
#         # Check if the file exists
#         if peptide_file_exists:
#             # Read the existing peptides from the file
#             with open('example_reanalysis_peptide.tsv', 'r') as existing_file:
#                 # Skip the header
#                 next(existing_file)
#                 for line in existing_file:
#                     peptide = line.split('\t')[0]  # Extract the peptide sequence (first column)
#                     unique_peptides_set.add(peptide)
#         print(f"Number of unique peptides added: {len(unique_peptides_set)}")
#         for chunk in pd.read_csv('example_reanalysis_spectrum.tsv', sep='\t', chunksize=chunk_size):
#             chunk['Peptide sequence'] = chunk.apply(get_peptide_sequence, axis=1)
#             counter = 0
#             total_peptides = len(chunk['Peptide sequence'].unique())

#             def update_counter(result):
#                 global counter
#                 counter += 1
#                 # print(f"Processed {counter}/{total_peptides} peptides")

#             with Pool(cpu_count()) as pool:
#                 peptides_to_process = [
#                     (peptide, chunk[chunk['Peptide sequence'] == peptide], peptideatlas_df)
#                     for peptide in chunk['Peptide sequence'].unique()
#                 ]
                
#                 results = []
#                 to_parallel_process = []
#                 not_parallel_process = []

#                 for peptide, peptide_data, pa_df in peptides_to_process:
#                     if peptide not in unique_peptides_set:
#                         #print(f"Appending New Peptide: {peptide}")
#                         unique_peptides_set.add(peptide)
#                         to_parallel_process.append((peptide, peptide_data, pa_df))
#                     else:
#                         #print(f"Processing Existing Peptide: {peptide}")
#                         not_parallel_process.append((peptide, peptide_data, pa_df))

#                 # Parallel process the new peptides
#                 if to_parallel_process:
#                     for result in pool.imap_unordered(process_peptide, to_parallel_process):
#                         results.append(result)
#                         update_counter(result)
#                         print(f"Processed {counter}/{total_peptides} peptides in chunk {chunk.index[0] // chunk_size + 1}")

#                 # Process the existing peptides
#                 for peptide, peptide_data, pa_df in not_parallel_process:
#                     results.append(process_existing_peptide(peptide, peptide_data, pa_df))
#                     update_counter(None)
#                     print(f"Processed {counter}/{total_peptides} peptides in chunk {chunk.index[0] // chunk_size + 1}")
                    
#                 for peptide_row in results:
#                     peptide_sequence = peptide_row['Peptide sequence']
                    
#                     # Read the current output file content
#                     output_file.seek(0)
#                     lines = output_file.readlines()
#                     print()
#                     # Check if the peptide is already in the file
#                     found = False
#                     for i, line in enumerate(lines):
#                         if line.startswith(peptide_sequence):
#                             found = True
#                             existing_data = line.strip().split('\t')
                            
#                             # Update the existing line with new data
#                             existing_data[3] = str(int(existing_data[3]) + peptide_row['Num_specs_both'])
#                             existing_data[4] = str(int(existing_data[4]) + peptide_row['Num_specs_MSGF'])
#                             existing_data[5] = str(int(existing_data[5]) + peptide_row['Num_specs_PA'])
#                             existing_data[6] = str(int(existing_data[6]) + peptide_row['PA_peptide'])
#                             existing_data[7] = str(int(existing_data[7]) + peptide_row['PA_psms'])
                            
#                             # Write the updated line back to the file
#                             lines[i] = '\t'.join(existing_data) + '\n'
#                             break
                    
#                     if not found:
#                         # Append the new peptide row to the file
#                         lines.append('\t'.join(map(str, peptide_row.values())) + '\n')
                    
#                     # Write the updated content back to the file
#                     output_file.seek(0)
#                     output_file.truncate()
#                     output_file.writelines(lines)

#                 pool.close()
#                 pool.join()

#             print(f'Processed chunk {chunk.index[0] // chunk_size + 1}')

#### with indexing

In [2]:
import pickle

# Load the data from dictionaries.pkl
with open('Human_proteome_dictionary_I_replaced_by_L.pkl', 'rb') as file:
    data = pickle.load(file)

# Extract the prefix and suffix dictionaries
prefix_dict = data["prefix_dict"]

# Print the dictionaries to verify
print("Prefix Dictionary (first 5):", dict(list(prefix_dict.items())[:5]))

print("Keys in Prefix Dictionary:", set(prefix_dict.keys()))


Prefix Dictionary (first 5): {'MDAA': [('A0A087WV00', 'DGKI', 1057), ('A6NMZ7', 'COL6A6', 300), ('O14901', 'KLF11', 426), ('O15085', 'ARHGEF11', 884), ('O43303', 'CCP110', 136), ('O43610', 'SPRY3', 288), ('O60239', 'SH3BP5', 455), ('O75311', 'GLRA3', 186), ('O75912', 'DGKI', 1065), ('P02533', 'KRT14', 201), ('P08514', 'ITGA2B', 374), ('P08779', 'KRT16', 200), ('P14868', 'DARS1', 484), ('P19012', 'KRT15', 194), ('P20142', 'PGC', 320), ('P20810', 'CAST', 556), ('P23415', 'GLRA1', 184), ('P23416', 'GLRA2', 173), ('P23771', 'GATA3', 408), ('P24298', 'GPT', 165), ('P35268', 'RPL22', 95), ('P41091', 'EIF2S3', 321), ('P45974', 'USP5', 366), ('P55291', 'CDH15', 814), ('Q02252', 'ALDH6A1', 457), ('Q02763', 'TEK', 275), ('Q04695', 'KRT17', 192), ('Q05823', 'RNASEL', 569), ('Q13064', 'MKRN3', 231), ('Q13324', 'CRHR2', 411), ('Q14674', 'ESPL1', 1567), ('Q5W0B1', 'OBI1', 197), ('Q6IE81', 'JADE1', 697), ('Q6NZY7', 'CDC42EP5', 42), ('Q6PGQ7', 'BORA', 182), ('Q6PJI9', 'WDR59', 566), ('Q7Z6Z7', 'HUWE1'

dictionary look like: Prefix Dictionary (first 5): {'MDAA': [('A0A087WV00', 'DGKI', 1057),('A6NMZ7', 'COL6A6', 300),...... if all 4mer form peptide can be found in same protein and the length as showed in example here for prefix and length is >= the actual length of the peptide, the peptide is found in exact match, if there are <= 4  4-mers not found, then its in saap match. 

spectrum df look like this:PeptideAtlas_USI	PeptideAtlas_peptide	PeptideAtlas_peptide_demod	Peptide_match	PeptideAtlas_charge	rowid	sequence	PSM_ID	accession	unique	database	database_version	search_engine	modifications	retention_time	charge	exp_mass_to_charge	calc_mass_to_charge	spectra_ref	pre	post	start	end	opt_global_valid	opt_global_invalid_reason	opt_global_AllGenes	opt_global_AllProteins	opt_global_CanonicalProteins	opt_global_DeNovoScore	opt_global_EValue	opt_global_FDR	opt_global_FragMethod	opt_global_IsotopeError	opt_global_MSGFScore	opt_global_NumPSMsForAllProteins	opt_global_NumPSMsForSpectrum	opt_global_NumPSMsForTopProtein	opt_global_NumPrecsForTopProtein	opt_global_NumSharedPSMsForTopProtein	opt_global_NumSharedPrecsForTopProtein	opt_global_NumUniquePSMsForTopProtein	opt_global_NumUniquePrecsForTopProtein	opt_global_OriginalFilepath	opt_global_PSM_sorting_score	opt_global_PeptideLength	opt_global_PrecQValue	opt_global_Precursor	opt_global_PrecursorError(ppm)	opt_global_Protein	opt_global_QValue	opt_global_SpecEValue	opt_global_SpecID	opt_global_Title	opt_global_TopCanonicalProtQValue	opt_global_TopCanonicalProtein	opt_global_TopGene	opt_global_TopGeneQValue	opt_global_TopGeneScore	opt_global_TopProtQValue	opt_global_TopProteinScore	opt_global_TopUniqueGene	opt_global_TopUniqueProtein	opt_global_UniqueGeneQValue	opt_global_UniqueProtQValue	opt_global_UnmodPep	opt_global_UnmodPepIL	opt_global_ambiguity_total_score	opt_global_collision_energy	opt_global_decoy	opt_global_filename	opt_global_first_second_unique_ratio	opt_global_first_unique_count	opt_global_first_unique_intensity	opt_global_kl_interpeak	opt_global_kl_strict	opt_global_kl_unstrict	opt_global_numberpsms	opt_global_ppm_error	opt_global_precursor_pep	opt_global_scan	opt_global_score	opt_global_second_unique_count	opt_global_second_unique_intensity	opt_global_sequence	opt_global_sorting_score	opt_global_spectrum_unique_key	opt_global_pass_threshold	opt_global_cv_MS:1002217_decoy_peptide	opt_global_cv_MS:1002354_PSM-level_q-value	#SpecFile	nativeID	mod_set	unique_PSM_ID	modified_sequence	variant	_dyn_#AllGenes	_dyn_#AllProteins	_dyn_#CanonicalProteins	_dyn_#DeNovoScore	_dyn_#EValue	_dyn_#FDR	_dyn_#FragMethod	_dyn_#IsotopeError	_dyn_#MSGFScore	_dyn_#NumPSMsForAllProteins	_dyn_#NumPSMsForSpectrum	_dyn_#NumPSMsForTopProtein	_dyn_#NumPrecsForTopProtein	_dyn_#NumSharedPSMsForTopProtein	_dyn_#NumSharedPrecsForTopProtein	_dyn_#NumUniquePSMsForTopProtein	_dyn_#NumUniquePrecsForTopProtein	_dyn_#OriginalFilepath	_dyn_#PSM_sorting_score	_dyn_#PeptideLength	_dyn_#PrecQValue	_dyn_#Precursor	_dyn_#PrecursorError(ppm)	_dyn_#Protein	_dyn_#QValue	_dyn_#SpecEValue	_dyn_#SpecID	_dyn_#Title	_dyn_#TopCanonicalProtQValue	_dyn_#TopCanonicalProtein	_dyn_#TopGene	_dyn_#TopGeneQValue	_dyn_#TopGeneScore	_dyn_#TopProtQValue	_dyn_#TopProteinScore	_dyn_#TopUniqueGene	_dyn_#TopUniqueProtein	_dyn_#UniqueGeneQValue	_dyn_#UniqueProtQValue	_dyn_#UnmodPep	_dyn_#UnmodPepIL	_dyn_#ambiguity_total_score	_dyn_#collision_energy	_dyn_#decoy	_dyn_#filename	_dyn_#first_second_unique_ratio	_dyn_#first_unique_count	_dyn_#first_unique_intensity	_dyn_#kl_interpeak	_dyn_#kl_strict	_dyn_#kl_unstrict	_dyn_#numberpsms	_dyn_#ppm_error	_dyn_#precursor_pep	_dyn_#scan	_dyn_#score	_dyn_#second_unique_count	_dyn_#second_unique_intensity	_dyn_#sequence	_dyn_#sorting_score	_dyn_#spectrum_unique_key	_dyn_#pass_threshold	_dyn_#cv_MS:1002217_decoy_peptide	_dyn_#cv_MS:1002354_PSM-level_q-value	valid	internalFilename	nativeID_index	nativeID_scan	nativeID_index_1-based	baseFilename	sequence_li	modified_sequence_li	AllGenes	AllProteins	CanonicalProteins	DeNovoScore	EValue	FDR	FragMethod	IsotopeError	MSGFScore	NumPSMsForAllProteins	NumPSMsForSpectrum	NumPSMsForTopProtein	NumPrecsForTopProtein	NumSharedPSMsForTopProtein	NumSharedPrecsForTopProtein	NumUniquePSMsForTopProtein	NumUniquePrecsForTopProtein	OriginalFilepath	PSM_sorting_score	PeptideLength	PrecQValue	Precursor	PrecursorError(ppm)	Protein	QValue	SpecEValue	SpecID	Title	TopCanonicalProtQValue	TopCanonicalProtein	TopGene	TopGeneQValue	TopGeneScore	TopProtQValue	TopProteinScore	TopUniqueGene	TopUniqueProtein	UniqueGeneQValue	UniqueProtQValue	UnmodPep	UnmodPepIL	ambiguity_total_score	collision_energy	decoy	filename	first_second_unique_ratio	first_unique_count	first_unique_intensity	kl_interpeak	kl_strict	kl_unstrict	numberpsms	ppm_error	precursor_pep	scan	score	second_unique_count	second_unique_intensity	sequence.1	sorting_score	spectrum_unique_key	pass_threshold	cv_MS:1002217_decoy_peptide	cv_MS:1002354_PSM-level_q-value	id
					29.0	VDDTQFVRFDSDAASQKMEPRAPW	29.0	tr|A0A1W2PSE7|A0A1W2PSE7_HUMAN							4.0			ms_run[5]:scan=33297					VALID		HLA-A	tr|A0A1W2PSE7|A0A1W2PSE7_HUMAN;tr|A0A0G2JI36|A0A0G2JI36_HUMAN;tr|Q5SUL5|Q5SUL5_HUMAN;tr|A0A0G2JL56|A0A0G2JL56_HUMAN;tr|A0A1W2PQD0|A0A1W2PQD0_HUMAN;tr|A0A0G2JIF2|A0A0G2JIF2_HUMAN		229.0	0.03459448	0.009884998061765	HCD	0.0	99.0	4338.0	1.0	40.0	11.0	40.0	11.0	0.0	0.0	MSV000096130/ccms_peak/RAW/20170317_QEh1_LC2_FaMa_ChCh_SA_HLApII_RA957_3_R1.mzML	1.46099319302198	24.0	0.0007213445863088	699.8391	10.727329	tr|Q5SUL5|Q5SUL5_HUMAN	0.009884998061765	1.1862006e-09	index=27322	Scan Number: 33297			HLA-A	0.0	4657.41082709346	0.0	71.1125831911783	HLA-A		0.0		VDDTQFVRFDSDAASQKMEPRAPW	VDDTQFVRFDSDAASQKMEPRAPW	-1.0	0.0	0.0	spec-00004.mgf	-1.0	-1.0	-1.0	-1.0	-1.0	-1.0	1.0	10.727329	VDDTQFVRFDSDAASQKMEPRAPW.4	33297	1.46099319302198	-1.0	-1.0	VDDTQFVRFDSDAASQKMEPRAPW.4	1.46099319302198	spec-00004.mgf:33297			0.009884998061765	MSV000096130/ccms_peak/RAW/20170317_QEh1_LC2_FaMa_ChCh_SA_HLApII_RA957_3_R1.mzML	scan=33297		scan=33297_VDDTQFVRFDSDAASQKMEPRAPW_null	VDDTQFVRFDSDAASQKMEPRAPW	VDDTQFVRFDSDAASQKMEPRAPW_4	HLA-A	tr|A0A1W2PSE7|A0A1W2PSE7_HUMAN;tr|A0A0G2JI36|A0A0G2JI36_HUMAN;tr|Q5SUL5|Q5SUL5_HUMAN;tr|A0A0G2JL56|A0A0G2JL56_HUMAN;tr|A0A1W2PQD0|A0A1W2PQD0_HUMAN;tr|A0A0G2JIF2|A0A0G2JIF2_HUMAN		229.0	0.03459448	0.009884998061765	HCD	0.0	99.0	4338.0	1.0	40.0	11.0	40.0	11.0	0.0	0.0	MSV000096130/ccms_peak/RAW/20170317_QEh1_LC2_FaMa_ChCh_SA_HLApII_RA957_3_R1.mzML	1.46099319302198	24.0	0.0007213445863088	699.8391	10.727329	tr|Q5SUL5|Q5SUL5_HUMAN	0.009884998061765	1.1862006e-09	index=27322	Scan Number: 33297			HLA-A	0.0	4657.41082709346	0.0	71.1125831911783	HLA-A		0.0		VDDTQFVRFDSDAASQKMEPRAPW	VDDTQFVRFDSDAASQKMEPRAPW	-1.0	0.0	0.0	spec-00004.mgf	-1.0	-1.0	-1.0	-1.0	-1.0	-1.0	1.0	10.727329	VDDTQFVRFDSDAASQKMEPRAPW.4	33297.0	1.46099319302198	-1.0	-1.0	VDDTQFVRFDSDAASQKMEPRAPW.4	1.46099319302198	spec-00004.mgf:33297			0.009884998061765	VALID	f.MSV000096130/ccms_peak/RAW/20170317_QEh1_LC2_FaMa_ChCh_SA_HLApII_RA957_3_R1.mzML	-1.0	33297.0	-1.0	20170317_QEh1_LC2_FaMa_ChCh_SA_HLApII_RA957_3_R1	VDDTQFVRFDSDAASQKMEPRAPW	VDDTQFVRFDSDAASQKMEPRAPW																																																																	
					34.0	EDLSSWTAADTAAQITQ	34.0	tr|S6AU73|S6AU73_HUMAN							2.0	

sequence is a non repeating set read from PeptideAtlas_peptide_demod and sequence column, and charge is return row['PeptideAtlas_charge'] if pd.notna(row['PeptideAtlas_charge']) else row['charge'], 

mutate every aa to all 20 posibilities, and by the ned return the Union of each peptide return for protein and genes

In [3]:
# Testing for one peptide
def find_exact_matches(peptide, dictionary):
    peptide = peptide.replace('I', 'L')
    fourmers = [peptide[i:i+4] for i in range(len(peptide) - 3)]
    matching_proteins = None
    matching_genes = None

    for fourmer in fourmers:
        if fourmer in dictionary:
            proteins = {entry[0] for entry in dictionary[fourmer]}
            genes = {entry[1] for entry in dictionary[fourmer]}
            if matching_proteins is None:
                matching_proteins = proteins
                matching_genes = genes
            else:
                matching_proteins &= proteins
                filtered_genes = {gene for gene in genes if gene != 'Unknown'}
                matching_genes &= filtered_genes
        else:
            return set(), set()  # If any 4-mer is not found, no exact match

    return matching_proteins if matching_proteins else set(), matching_genes if matching_genes else set()

# Function to find SAAP matches (one amino acid difference)
def find_saap_matches(peptide, dictionary):
    peptide = peptide.replace('I', 'L')
    mutated_proteins = set()
    mutated_genes = set()

    for i in range(len(peptide)):
        for aa in "ACDEFGHKLMNPQRSTVWY":  # Don't need to include 'I' here, because we already replaced 'I' with 'L'
            mutated_peptide = peptide[:i] + aa + peptide[i+1:]
            proteins, genes = find_exact_matches(mutated_peptide, dictionary)
            mutated_proteins.update(proteins)
            # Filter out 'Unknown' from genes
            filtered_genes = {gene for gene in genes if gene != 'Unknown'}
            mutated_genes.update(filtered_genes)

    return mutated_proteins, mutated_genes

In [4]:
import pandas as pd
import time
import os

# Load the spectrum data to get unique peptides
spectrum_file = 'example_reanalysis_spectrum.tsv'
spectrum_df = pd.read_csv(spectrum_file, sep='\t')

# Extract unique peptides
peptide_df = spectrum_df[['PeptideAtlas_peptide_demod', 'sequence', 'PeptideAtlas_charge', 'charge','accession']].copy()
peptide_df['Peptide sequence'] = peptide_df.apply(
    lambda row: row['PeptideAtlas_peptide_demod'] if pd.notna(row['PeptideAtlas_peptide_demod']) else row['sequence'], axis=1
)
peptide_df['Peptide charge'] = peptide_df.apply(
    lambda row: row['PeptideAtlas_charge'] if pd.notna(row['PeptideAtlas_charge']) else row['charge'], axis=1
)
# Extract protein identifiers from the 'accession' column, handling both pipe-separated and non-pipe cases
peptide_df['Protein identifier'] = peptide_df['accession'].apply(
    lambda acc: ';'.join([
        a.split('|')[1] if isinstance(a, str) and '|' in a else str(a)
        for a in str(acc).split(';') if a and a != 'nan'
    ]) if pd.notna(acc) else ''
)
# Group by 'Peptide sequence' and 'Peptide charge', stacking all protein identifiers (deduplicated) with ';'
peptide_df = (
    peptide_df
    .groupby(['Peptide sequence', 'Peptide charge'], as_index=False)
    .agg({'Protein identifier': lambda x: ';'.join(sorted(set(';'.join(x).split(';')) - {''}))})
)

output_file = "example_reanalysis_peptide.tsv"
if os.path.exists(output_file):
    existing = pd.read_csv(output_file, sep='\t')
    done_peptides = set(zip(existing['Peptide sequence'], existing['Peptide charge']))
    peptide_df = peptide_df[~peptide_df.set_index(['Peptide sequence', 'Peptide charge']).index.isin(done_peptides)]
    mode = 'a'
    header = False
else:
    mode = 'w'
    header = True

if not peptide_df.empty:
    total_peptides = len(peptide_df)
    start_time = time.time()
    results = []
    for index, row in peptide_df.iterrows():
        peptide = row['Peptide sequence']

        # Find exact matches
        exact_matches_proteins, exact_matches_genes = find_exact_matches(peptide, prefix_dict)
        num_proteins = len(exact_matches_proteins)
        list_proteins = ';'.join(exact_matches_proteins) if exact_matches_proteins else 'None'
        num_genes = len(exact_matches_genes)
        list_genes = ';'.join(exact_matches_genes) if exact_matches_genes else 'None'

        # Find SAAP matches
        saap_matches_proteins, saap_matches_genes = find_saap_matches(peptide, prefix_dict)
        num_proteins_saap = len(saap_matches_proteins)
        list_proteins_saap = ';'.join(saap_matches_proteins) if saap_matches_proteins else 'None'
        num_genes_saap = len(saap_matches_genes)
        list_genes_saap = ';'.join(saap_matches_genes) if saap_matches_genes else 'None'

        peptide_row = {
            'Peptide sequence': peptide,
            'Peptide charge': int(row['Peptide charge']),
            'Protein identifier': row['Protein identifier'],
            'Num_specs_both': 0,
            'Num_specs_MSGF': 0,
            'Num_specs_PA': 0,
            'PA_peptide': 0,
            'PA_psms': 0,
            'Num_proteins': int(num_proteins),
            'List_proteins': list_proteins,
            'Num_genes': int(num_genes),
            'List_genes': list_genes,
            'Num_proteins_saap': int(num_proteins_saap),
            'List_proteins_saap': list_proteins_saap,
            'Num_genes_saap': int(num_genes_saap),
            'List_genes_saap': list_genes_saap
        }
        results.append(peptide_row)

        # Print progress
        if (index + 1) % 10 == 0 or (index + 1) == total_peptides:
            elapsed_time = time.time() - start_time
            percentage = ((index + 1) / total_peptides) * 100
            estimated_time_left = (elapsed_time / (index + 1)) * (total_peptides - (index + 1))
            estimated_hours = int(estimated_time_left // 3600)
            estimated_minutes = int((estimated_time_left % 3600) // 60)
            estimated_seconds = int(estimated_time_left % 60)
            print(f"\rProcessed {index + 1}/{total_peptides} peptides ({percentage:.2f}%). Elapsed time: {elapsed_time:.2f}s. Estimated time left: {estimated_hours} hours {estimated_minutes} minutes {estimated_seconds} seconds.", end="")

    pd.DataFrame(results).to_csv(output_file, sep="\t", index=False, mode=mode, header=header)
else:
    print("No new peptides to process.")

C:\Users\xuech\AppData\Local\Temp\ipykernel_29220\116921559.py:7: DtypeWarning: Columns (0,1,2,7,9,10,11,15,26,32,35,36,38,39,44,45,48,49,55,65,70,72,73,74,81,82,83,84,85,86,90,91,92,96,107,113,116,117,119,120,125,126,129,130,136,140,147,152,154,155,160,161,162,165,166,167) have mixed types. Specify dtype option on import or set low_memory=False.
  spectrum_df = pd.read_csv(spectrum_file, sep='\t')


Processed 26180/452 peptides (5792.04%). Elapsed time: 710.71s. Estimated time left: -1 hours 48 minutes 21 seconds.

In [ ]:
# Query the protein from 'accession' in spectrum_df and add to peptide_df as 'Protein identifier'
import pandas as pd

# Load the peptide and spectrum data
peptide_file = "example_peptide_reanalysis_updated.tsv"
spectrum_file = "example_reanalysis_spectrum.tsv"

peptide_df = pd.read_csv(peptide_file, sep="\t")
spectrum_df = pd.read_csv(spectrum_file, sep="\t")

# For each peptide, find all unique protein identifiers from spectrum_df and update the peptide_df
protein_ids_all = []
total = len(peptide_df)
for idx, row in peptide_df.iterrows():
    peptide = row['Peptide sequence']
    # Find all matching rows in spectrum_df
    matches = spectrum_df[
        (spectrum_df['PeptideAtlas_peptide_demod'] == peptide) | 
        (spectrum_df['sequence'] == peptide)
    ]
    # Extract unique protein identifiers from 'accession' column
    protein_ids = matches['accession'].dropna().unique()
    # Parse to get only the protein accession (middle part if pipe-separated)
    protein_ids = [
        acc.split('|')[1] if isinstance(acc, str) and '|' in acc else acc
        for acc in protein_ids
    ]
    # Join all unique protein ids with ';'
    protein_ids_str = ";".join(sorted(set(protein_ids))) if protein_ids else ""
    protein_ids_all.append(protein_ids_str)
    # Print progress
    if (idx + 1) % 10 == 0 or (idx + 1) == total:
        percent = 100 * (idx + 1) / total
        print(f"\rProcessing: {percent:.2f}% ({idx + 1}/{total})", end='', flush=True)
print()  # for newline after progress

# Add/replace the column in peptide_df
peptide_df['Protein identifier'] = protein_ids_all

# Save the updated dataframe
peptide_df.to_csv(peptide_file, sep="\t", index=False)

C:\Users\xuech\AppData\Local\Temp\ipykernel_1036\1586994430.py:9: DtypeWarning: Columns (0,1,2,7,9,10,11,15,26,32,35,36,38,39,44,45,48,49,55,65,70,72,73,74,81,82,83,84,85,86,90,91,92,96,107,113,116,117,119,120,125,126,129,130,136,140,147,152,154,155,160,161,162,165,166,167) have mixed types. Specify dtype option on import or set low_memory=False.
  spectrum_df = pd.read_csv(spectrum_file, sep="\t")


Processing: 100.00% (26209/26209)


PermissionError: [Errno 13] Permission denied: 'example_peptide_reanalysis_updated.tsv'

In [ ]:
# peptide_df.to_csv('example_peptide_reanalysis_updated1.tsv', sep="\t", index=False)

In [3]:
import pandas as pd

# Load the peptide file
peptide_df = pd.read_csv(peptide_file, sep="\t")

# Copy 'Protein identifier' column to 'Protein identifier MSGF'
peptide_df['Protein identifier MSGF'] = peptide_df['Protein identifier']

# Save the updated dataframe
peptide_df.to_csv(peptide_file, sep="\t", index=False)

In [ ]:

# # Define the peptide to search
# peptide = "QARERAEADVASLNR"

# # Replace 'I' with 'L' in the peptide for consistency
# peptide = peptide.replace('I', 'L')
# # Use find_exact_matches to find proteins and genes
# exact_matches_proteins, exact_matches_genes = find_exact_matches(peptide, prefix_dict)
# # Update the variables with the results
# num_proteins = len(exact_matches_proteins)
# num_genes = len(exact_matches_genes)

# # Use find_saap_matches to find proteins and genes with one amino acid difference
# saap_matches_proteins, saap_matches_genes = find_saap_matches(peptide, prefix_dict)

# # Update the variables with the results
# num_proteins_saap = len(saap_matches_proteins)
# num_genes_saap = len(saap_matches_genes)

# # Print out the proteins and genes for exact matches
# print("Exact Match Proteins:", exact_matches_proteins)
# print("Exact Match Genes:", exact_matches_genes)

# # Print out the proteins and genes for SAAP matches
# print("SAAP Match Proteins:", saap_matches_proteins)
# print("SAAP Match Genes:", saap_matches_genes)



Exact Match Proteins: {'P07951-3'}
Exact Match Genes: {'Unknown', 'TPM2'}
SAAP Match Proteins: {'P07951-3'}
SAAP Match Genes: {'TPM2'}


#### fix peptide number

In [7]:
import pandas as pd
import re

# Load the peptide and spectrum data
peptide_data_path = "example_reanalysis_peptide.tsv"
spectrum_data_path = "example_reanalysis_spectrum.tsv"
peptideatlas_df = pd.read_csv('PeptideAtlas_peptides.tsv', sep="\t")
usi_file = 'all_usi.txt'

with open(usi_file, 'r') as file:
    peptides_pa = [re.sub(r'\[.*?\]', '', line.strip().split(':')[-1].split('/')[0].replace('-', '')) for line in file if len(line.strip().split(':')) == 6]

peptide_df = pd.read_csv(peptide_data_path, sep="\t")
spectrum_df = pd.read_csv(spectrum_data_path, sep="\t")


def get_protein_id_from_pa(row):
    peptide_sequence = row['Peptide sequence']
    matching_rows = peptideatlas_df[peptideatlas_df.iloc[:, 5] == peptide_sequence]
    if not matching_rows.empty:
        return matching_rows.iloc[0, 0]  # Return the protein ID from column 0
    return None
output_df = pd.DataFrame()

# Function to update the specified columns
def update_peptide_data(peptide_df, spectrum_df):
    # Iterate through each peptide in the peptide dataframe
    pa_observations_df = pd.read_csv('PA_observations.csv')
    for index, row in peptide_df.iterrows():
        # Print progress update
        if index % 100 == 0:
            print(f"Processing peptide {index+1}/{len(peptide_df)} ({(index+1)/len(peptide_df)*100:.1f}%)")
        peptide_sequence = row['Peptide sequence']
        
        # Filter spectrum data for the current peptide
        spectrum_subset = spectrum_df[(spectrum_df['sequence'] == peptide_sequence) | (spectrum_df['PeptideAtlas_peptide_demod'] == peptide_sequence)]
        
        # Update the columns
        num_specs_both = len(spectrum_subset[(pd.notna(spectrum_subset['PeptideAtlas_USI'])) & (pd.notna(spectrum_subset['sequence']))])
        num_specs_msgf = len(spectrum_subset[(pd.isna(spectrum_subset['PeptideAtlas_USI'])) & (pd.notna(spectrum_subset['sequence']))])
        num_specs_pa = len(spectrum_subset[(pd.notna(spectrum_subset['PeptideAtlas_USI'])) & (pd.isna(spectrum_subset['sequence']))])
        pa_peptide = 0

        if peptide_sequence in pa_observations_df['Sequence'].values:
            pa_peptide = 1
        # Count PSMs from all_usi.txt
        pa_psms = peptides_pa.count(peptide_sequence)
        # try:
        #     with open('all_usii.txt', 'r') as usi_file:
        #         for line in usi_file:
        #             usi = line.strip()
        #             # Extract and demodify the peptide sequence from the USI
        #             if ":" not in usi or "/" not in usi:
        #                 continue
                    
        #             # Extract sequence part between last colon and slash
        #             parts = usi.split(':')
        #             if len(parts) < 2:
        #                 continue
                    
        #             seq_part = parts[-1].split('/')[0]
                    
        #             # Remove modifications (text in square brackets)
        #             demod_seq = re.sub(r'\[.*?\]', '', seq_part)
                    
        #             # If peptide matches the current peptide sequence
        #             if demod_seq == peptide_sequence:
        #                 pa_psms += 1
        # except FileNotFoundError:
        # pa_psms = len(spectrum_subset[spectrum_subset['sequence'] == peptide_sequence])
        
        # Update the peptide dataframe
        
        protein_identifier = row['Protein identifier']
        if isinstance(protein_identifier, str):
            peptide_df.at[index, 'Protein identifier MSGF'] = protein_identifier.split('-')[0]
        else:
            peptide_df.at[index, 'Protein identifier MSGF'] = None
        peptide_df.at[index, 'Protein identifier PA'] = ";".join(peptideatlas_df[peptideatlas_df.iloc[:, 5] == peptide_sequence].iloc[:, 0].dropna().unique())
        peptide_df.at[index, 'Num_specs_both'] = num_specs_both
        peptide_df.at[index, 'Num_specs_MSGF'] = num_specs_msgf
        peptide_df.at[index, 'Num_specs_PA'] = num_specs_pa
        peptide_df.at[index, 'PA_peptide'] = pa_peptide
        peptide_df.at[index, 'PA_psms'] = pa_psms

    return peptide_df

# Update the peptide data
updated_peptide_df = update_peptide_data(peptide_df, spectrum_df)

# Save the updated dataframe
updated_peptide_df.to_csv("example_peptide_reanalysis_updated.tsv", sep="\t", index=False)

C:\Users\xuech\AppData\Local\Temp\ipykernel_29220\1316664837.py:14: DtypeWarning: Columns (0,1,2,7,9,10,11,15,26,32,35,36,38,39,44,45,48,49,55,65,70,72,73,74,81,82,83,84,85,86,90,91,92,96,107,113,116,117,119,120,125,126,129,130,136,140,147,152,154,155,160,161,162,165,166,167) have mixed types. Specify dtype option on import or set low_memory=False.
  spectrum_df = pd.read_csv(spectrum_data_path, sep="\t")


Processing peptide 1/26209 (0.0%)
Processing peptide 101/26209 (0.4%)
Processing peptide 201/26209 (0.8%)
Processing peptide 301/26209 (1.1%)
Processing peptide 401/26209 (1.5%)
Processing peptide 501/26209 (1.9%)
Processing peptide 601/26209 (2.3%)
Processing peptide 701/26209 (2.7%)
Processing peptide 801/26209 (3.1%)
Processing peptide 901/26209 (3.4%)
Processing peptide 1001/26209 (3.8%)
Processing peptide 1101/26209 (4.2%)
Processing peptide 1201/26209 (4.6%)
Processing peptide 1301/26209 (5.0%)
Processing peptide 1401/26209 (5.3%)
Processing peptide 1501/26209 (5.7%)
Processing peptide 1601/26209 (6.1%)
Processing peptide 1701/26209 (6.5%)
Processing peptide 1801/26209 (6.9%)
Processing peptide 1901/26209 (7.3%)
Processing peptide 2001/26209 (7.6%)
Processing peptide 2101/26209 (8.0%)
Processing peptide 2201/26209 (8.4%)
Processing peptide 2301/26209 (8.8%)
Processing peptide 2401/26209 (9.2%)
Processing peptide 2501/26209 (9.5%)
Processing peptide 2601/26209 (9.9%)
Processing pe

## Protein level

In [4]:
import pandas as pd

# Load the data
peptide_file = 'example_peptide_reanalysis_updated.tsv'
protein_file = 'PeptideAtlas_proteins_not_in_MassIVE.tsv'

peptide_df = pd.read_csv(peptide_file, sep='\t')
protein_df = pd.read_csv(protein_file, sep='\t')

# Initialize the result dataframe
result_columns = [
    'Protein', 'Num_peptides_both', 'Num_peptides_MSGF', 'Num_peptides_PA',
    'Num_unique_both', 'Num_unique_MSGF', 'Num_unique_PA',
    'Num_specs_both', 'Num_specs_MSGF', 'Num_specs_PA'
]
result_df = pd.DataFrame(columns=result_columns)

# Iterate over each protein
result_list = []
for protein in protein_df['nextprot_accession']:
    # Filter peptides that match the current protein
    matched_peptides = peptide_df[
        peptide_df['Protein identifier MSGF'].str.contains(protein, na=False) |
        peptide_df['Protein identifier PA'].str.contains(protein, na=False)
    ]
    
    # Calculate the required counts and sums
    num_peptides_both = len(matched_peptides[matched_peptides['Num_specs_both'] > 0])
    num_peptides_MSGF = len(matched_peptides[matched_peptides['Num_specs_MSGF'] > 0])
    num_peptides_PA = len(matched_peptides[matched_peptides['Num_specs_PA'] > 0])
    
    num_unique_both = len(matched_peptides[(matched_peptides['Num_specs_both'] > 0) & (matched_peptides['Num_genes_saap'] == 1)])
    num_unique_MSGF = len(matched_peptides[(matched_peptides['Num_specs_MSGF'] > 0) & (matched_peptides['Num_genes_saap'] == 1)])
    num_unique_PA = len(matched_peptides[(matched_peptides['Num_specs_PA'] > 0) & (matched_peptides['Num_genes_saap'] == 1)])
    
    num_specs_both = matched_peptides['Num_specs_both'].sum()
    num_specs_MSGF = matched_peptides['Num_specs_MSGF'].sum()
    num_specs_PA = matched_peptides['Num_specs_PA'].sum()
    
    # Append the results to the result list
    result_list.append({
        'Protein': protein,
        'Num_peptides_both': num_peptides_both,
        'Num_peptides_MSGF': num_peptides_MSGF,
        'Num_peptides_PA': num_peptides_PA,
        'Num_unique_both': num_unique_both,
        'Num_unique_MSGF': num_unique_MSGF,
        'Num_unique_PA': num_unique_PA,
        'Num_specs_both': num_specs_both,
        'Num_specs_MSGF': num_specs_MSGF,
        'Num_specs_PA': num_specs_PA
    })

# Convert the result list to a dataframe
result_df = pd.concat([pd.DataFrame([result]) for result in result_list], ignore_index=True)

# Save the result to a new file in TSV format
result_df.to_csv('example_protein_reanalysis.tsv', sep='\t', index=False)

#Check ambiguity filterintg:


In [ ]:
# # Define the two lists
# list1 = [
#     "A0A0C4DH55", "Q6UX40", "O95406", "Q9Y284", "A0A075B6K0", "Q8N112", "A0A0C4DH41", "Q96FC9", "Q96DS6", "Q96MN9",
#     "Q6UWI2", "Q5UAW9", "Q8WV19", "Q7Z4L0", "Q5T1S8", "Q08AH3", "A0A1B0GUW7", "A0A2R8Y4L2", "P01772", "Q02161",
#     "P0CG38", "P18089", "P17538", "B6A8C7", "O95484", "O75360", "Q8N131", "Q86T20", "A0A2R8YCJ5", "P49682"
# ]

# list2 = [
#     "Q96MN9", "P01742", "A0A087WSY4", "A0A0C4DH68", "Q6UX40", "P0CG39", "Q6UWI2", "Q5UAW9", "Q8WV19", "P68104",
#     "A0A1B0GUW7", "A0A2R8Y4L2", "P01599", "Q8N112", "A0A0C4DH41", "A0A0C4DH43", "A0A0C4DH72", "P01709", "P01624",
#     "P01772", "A0A075B6H8", "P0CG38", "P18089", "O95484", "O75360", "Q96DS6", "P62834", "Q8N131", "A0A0B4J2H0",
#     "A0A2R8YCJ5", "A0A0C4DH69", "P49682", "Q9Y4E1"
# ]

# # Convert to sets
# set1 = set(list1)
# set2 = set(list2)

# # Items in list1 but not in list2
# diff1 = sorted(set1 - set2)
# # Items in list2 but not in list1
# diff2 = sorted(set2 - set1)

# print("Items in list1 but not in list2:")
# print(diff1)
# print("\nItems in list2 but not in list1:")
# print(diff2)

Items in list1 but not in list2:
['A0A075B6K0', 'A0A0C4DH55', 'B6A8C7', 'O95406', 'P17538', 'Q02161', 'Q08AH3', 'Q5T1S8', 'Q7Z4L0', 'Q86T20', 'Q96FC9', 'Q9Y284']

Items in list2 but not in list1:
['A0A075B6H8', 'A0A087WSY4', 'A0A0B4J2H0', 'A0A0C4DH43', 'A0A0C4DH68', 'A0A0C4DH69', 'A0A0C4DH72', 'P01599', 'P01624', 'P01709', 'P01742', 'P0CG39', 'P62834', 'P68104', 'Q9Y4E1']


: 

In [ ]:
# import pandas as pd
# import time

# filename = 'ambiguity_merged.tsv'
# target = '01781_F03_P018699_S00_N22_R1'
# found = False

# # Get total number of lines (excluding header)
# with open(filename, 'r') as f:
#     total_lines = sum(1 for _ in f) - 1

# chunksize = 10000
# lines_checked = 0
# for chunk in pd.read_csv(filename, sep='\t', chunksize=chunksize, low_memory=False):
#     mask = chunk.apply(lambda row: row.astype(str).str.contains(target, na=False).any(), axis=1)
#     matching_rows = chunk[mask]
#     lines_checked += len(chunk)
#     percent = (lines_checked / total_lines) * 100
#     if lines_checked == chunksize:
#         start_time = time.time()
#     elif lines_checked > chunksize:
#         elapsed = time.time() - start_time
#         est_total = elapsed / (lines_checked / total_lines)
#         est_left = est_total - elapsed
#         hrs, rem = divmod(est_left, 3600)
#         mins, secs = divmod(rem, 60)
#         print(f"\rChecked {lines_checked} / {total_lines} lines... ({percent:.2f}%) | Estimated time left: {int(hrs)}h {int(mins)}m {int(secs)}s", end='')
#     else:
#         print(f"\rChecked {lines_checked} / {total_lines} lines... ({percent:.2f}%)", end='')

#     if not matching_rows.empty:
#         print(matching_rows)
#         found = True
#         break

# if not found:
#     print("\nnot found")

Checked 18550884 / 18550884 lines... (100.00%) | Estimated time left: 0h 0m 0ss
not found


In [1]:
import pandas as pd

# List of proteins to filter (as provided)
protein_list = [
    "Q14409", "A0A0K0K1C4", "A0M8Q6", "P0DJD3", "P06310", "A0A075B6R2", "Q96MN9", "P01700", "P01742", "P0DI81",
    "P22392", "A0A087WSY4", "P61326", "Q9BRT8", "A0A0C4DH68", "A0A075B6K2", "A0A0C4DH55", "A0A5B6", "Q15413",
    "Q6UX40", "P01615", "Q5EG05", "A0A075B6W5", "A0A075B6I7", "P69892", "Q6IS14", "P68133", "Q5VTL7", "P0CG39",
    "P0C0S5", "P0DMV8", "Q16617", "O95406", "Q6UWI2", "P0DP23", "O95626", "Q5UAW9", "P0CG47", "Q96L21", "Q9BUW7",
    "Q9H1U9", "Q8WV19", "Q56UQ5", "Q7Z4L0", "A0A075B767", "P35212", "Q5T1S8", "P47813", "P68104", "Q08AH3", "B0FP48",
    "Q9Y284", "A0A1B0GUW7", "P06331", "A0A075B6K0", "A0A2R8Y4L2", "P01599", "Q8N112", "A0A0C4DH41", "Q96IZ6",
    "A0A0C4DH43", "Q6P1Q9", "P0DPH7", "Q49MI3", "Q86VW1", "A0A0C4DH72", "A0A0C4DH35", "A0A0A0MRZ8", "P01614",
    "P04430", "P01824", "P01709", "P01624", "B2RPK0", "Q96FC9", "A0A2R8Y619", "Q6P461", "P01772", "Q6ZUT3", "Q02161",
    "A0A087WSY6", "A0A075B6S6", "P62837", "Q9NUN7", "P01768", "Q8NFI4", "A0A075B6N1", "A0A075B6H7", "P01825",
    "A0A075B6H8", "Q6S8J3", "A0A075B6R9", "P0CG38", "P18089", "P0DME0", "Q13885", "P17538", "O14604", "P59666",
    "B6A8C7", "P09341", "P98187", "P13164", "P50502", "O95484", "P68431", "P68371", "O75360", "P20231", "Q96DS6",
    "A8K0Z3", "Q6UXB3", "Q9H0U4", "P0CW19", "Q9BQ87", "P0DTL6", "Q8IZF0", "P62834", "Q7Z602", "A1L157", "Q8IWL1",
    "Q8IZP2", "A0A0C4DH42", "P01767", "P01593", "Q71UI9", "Q8N131", "A6NK59", "Q9BY64", "A0A1B0GUS4", "A0A0C4DH25",
    "A0A0C4DH73", "P61224", "A0A0C4DH34", "P01601", "A0A0B4J2H0", "Q9UQ74", "P61254", "Q147U7", "P04433", "Q9Y6W8",
    "A0A0C4DH36", "P32189", "P01597", "A0A0A6YYC5", "Q96IZ2", "A0A0B4J1X8", "Q86T20", "O43374", "A0A2R8YCJ5",
    "Q8TCE9", "Q9BVA1", "A0A0C4DH69", "P0CG34", "P86790", "P49682", "P0DTL5", "Q4G0N0", "P0DMM9", "A0A0A0MT36",
    "Q9Y4E1", "P0DSN7", "P01611", "Q5VYV0", "A5A3E0", "P0DN37", "B3EWG3", "Q8TCE6", "P15515", "Q99879", "P02538",
    "P47985", "O43423", "Q6PEY2", "Q3KPI0", "Q9BXR5", "Q96JQ5", "Q8N138"
]

# Load the protein-level results
protein_level_file = 'example_protein_reanalysis.tsv'
protein_df = pd.read_csv(protein_level_file, sep='\t')

# Filter for proteins in the provided list
filtered_protein_df = protein_df[protein_df['Protein'].isin(protein_list)]

# Save the filtered dataframe
filtered_protein_df.to_csv('filtered_example_protein_reanalysis.tsv', sep='\t', index=False)

# Optionally display the filtered dataframe
filtered_protein_df

,Protein,Num_peptides_both,Num_peptides_MSGF,Num_peptides_PA,Num_unique_both,Num_unique_MSGF,Num_unique_PA,Num_specs_both,Num_specs_MSGF,Num_specs_PA
8,Q14409,0,1,1,0,0,0,0.0,2.0,1.0
19,A0A0K0K1C4,0,0,0,0,0,0,0.0,0.0,0.0
22,A0M8Q6,0,11,0,0,0,0,0.0,226.0,0.0
25,P0DJD3,0,0,1,0,0,0,0.0,0.0,1.0
26,P06310,3,34,29,2,0,2,6.0,1919.0,88.0
...,...,...,...,...,...,...,...,...,...,...
1001,Q6PEY2,1,21,0,0,1,0,1.0,3346.0,0.0
1004,Q3KPI0,2,0,2,2,0,2,2.0,0.0,4.0
1012,Q9BXR5,0,3,1,0,2,0,0.0,3.0,1.0
1021,Q96JQ5,1,2,0,1,2,0,1.0,22.0,0.0
